In [ ]:
!pip install -U -q google-generativeai playwright beautifulsoup4 nest_asyncio
!playwright install
!playwright install-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 8.2 MB/s eta 0:00:00
(node:594) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 17.6s164.7 MiB [] 0% 10.0s164.7 MiB [] 0% 6.4s164.7 MiB [] 2% 3.1s164.7 MiB [] 3% 2.4s164.7 MiB [] 4% 2.2s164.7 MiB [] 5% 2.1s164.7 MiB [] 6% 1.8s164.7 MiB [] 7% 1.7s164.7 MiB [] 9% 1.6s164.7 MiB [] 10% 1.6s164.7 MiB [] 11% 1.5s164.7 MiB [] 13% 1.4s164.7 MiB [] 14% 1.3s164.7 MiB [] 15% 1.3s164.7 MiB [] 17% 1.3s164.7 MiB [] 18% 1.2s164.7 MiB [] 19% 1.2s164.7 MiB [] 21% 1.2s164.7 MiB [] 22% 1.1s164.7 MiB [] 24% 1.1s164.7 MiB [] 26% 1.0s164.7 MiB [] 27% 1.0s164.7 MiB [] 29% 0.9s164.7 MiB [] 29% 1.0s164.7

In [ ]:
import os
import re
import json
import asyncio
import nest_asyncio
import requests
from collections import Counter
from urllib.parse import urlparse, urljoin

import google.generativeai as genai
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from google.colab import userdata

nest_asyncio.apply()
os.environ.pop("GOOGLE_GENAI_USE_VERTEXAI", None)  # ensure not routing to Vertex

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [ ]:
class Universal_GEO_Auditor:
    def __init__(self, api_key):
        self.model_name = "gemini-2.5-pro"
        genai.configure(api_key=api_key)

        self.headers_google = {
            "User-Agent": (
                "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) "
                "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/W.X.Y.Z Mobile Safari/537.36 "
                "(compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
            )
        }

    # -------------------------
    # CONTENT CLEANING/CHUNKING
    # -------------------------
    def _clean_text(self, html):
        soup = BeautifulSoup(html, "html.parser")
        for tag in soup(["script", "style", "nav", "footer", "aside"]):
            tag.extract()
        return soup.get_text(separator=" ", strip=True)

    def _simulate_rag_chunking(self, text, chunk_size=1200, overlap=200):
        chunks = []
        start = 0
        while start < len(text):
            chunks.append(text[start:start + chunk_size])
            start += (chunk_size - overlap)
        return chunks

    # -------------------------
    # LINK EXTRACTION (for Service.url)
    # -------------------------
    def _extract_internal_links(self, html: str, base_url: str):
        soup = BeautifulSoup(html, "html.parser")
        base_domain = urlparse(base_url).netloc

        links = set()
        for a in soup.select("a[href]"):
            href = (a.get("href") or "").strip()
            if not href or href.startswith("#") or href.startswith("mailto:") or href.startswith("tel:"):
                continue
            abs_url = urljoin(base_url, href).split("#")[0].rstrip("/")
            if urlparse(abs_url).netloc == base_domain:
                links.add(abs_url)
        return sorted(links)

    def _candidate_service_urls(self, links):
        scored = []
        for u in links:
            path = urlparse(u).path.lower()
            score = 0
            if "/services/" in path: score += 6
            if "/service/" in path: score += 5
            if "/geo" in path: score += 3
            if "/seo" in path: score += 2
            if "/ppc" in path: score += 2
            if "/content" in path: score += 2
            if "/local" in path: score += 2
            if score > 0:
                scored.append((score, u))
        scored.sort(reverse=True, key=lambda x: x[0])
        return [u for _, u in scored[:30]]

    # -------------------------
    # SCHEMA AUDIT HELPERS
    # -------------------------
    def _extract_jsonld_blocks(self, html: str):
        soup = BeautifulSoup(html, "html.parser")
        return [s.get_text(strip=True) for s in soup.select('script[type="application/ld+json"]') if s.get_text(strip=True)]

    def _parse_jsonld_blocks(self, raw_blocks):
        parsed, errors = [], []
        for i, raw in enumerate(raw_blocks):
            try:
                obj = json.loads(raw)
                if isinstance(obj, dict):
                    parsed.append(obj)
                elif isinstance(obj, list):
                    parsed.extend(obj)
                else:
                    errors.append({"block": i, "error": f"Unexpected JSON root type: {type(obj)}", "preview": raw[:280]})
            except Exception as e:
                errors.append({"block": i, "error": str(e), "preview": raw[:280]})
        return parsed, errors

    def _collect_types(self, parsed_objects):
        types = []

        def walk(node):
            if isinstance(node, dict):
                t = node.get("@type")
                if isinstance(t, str):
                    types.append(t)
                elif isinstance(t, list):
                    types.extend([x for x in t if isinstance(x, str)])
                for v in node.values():
                    walk(v)
            elif isinstance(node, list):
                for x in node:
                    walk(x)

        for obj in parsed_objects:
            walk(obj)

        return Counter(types)

    def _indexability_signals(self, html: str):
        soup = BeautifulSoup(html, "html.parser")
        robots = soup.find("meta", attrs={"name": re.compile("^robots$", re.I)})
        canonical = soup.find("link", rel=lambda x: x and "canonical" in x.lower())
        return {
            "robots_meta": robots.get("content") if robots else None,
            "canonical": canonical.get("href") if canonical else None
        }

    # -------------------------
    # FETCH HTML (server vs rendered)
    # -------------------------
    async def _fetch_rendered_html(self, url: str):
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.set_extra_http_headers(self.headers_google)
            try:
                await page.goto(url)
                await page.wait_for_load_state("networkidle")
                html = await page.content()
                await browser.close()
                return html
            except Exception as e:
                await browser.close()
                print(f"❌ Render fetch error {url}: {e}")
                return ""

    def _fetch_server_html(self, url: str):
        try:
            r = requests.get(url, headers=self.headers_google, timeout=30)
            return r.text
        except Exception as e:
            print(f"❌ Server fetch error {url}: {e}")
            return ""

    def schema_audit_server(self, url: str):
        html_server = self._fetch_server_html(url)
        blocks = self._extract_jsonld_blocks(html_server)
        parsed, errors = self._parse_jsonld_blocks(blocks)
        types = self._collect_types(parsed)
        signals = self._indexability_signals(html_server)
        return {
            "blocks_count": len(blocks),
            "errors_count": len(errors),
            "types": dict(types.most_common(50)),
            "signals": signals,
            "errors_preview": errors[:3],
        }

    async def schema_audit_rendered(self, url: str):
        html = await self._fetch_rendered_html(url)
        blocks = self._extract_jsonld_blocks(html)
        parsed, errors = self._parse_jsonld_blocks(blocks)
        types = self._collect_types(parsed)
        signals = self._indexability_signals(html)
        return {
            "blocks_count": len(blocks),
            "errors_count": len(errors),
            "types": dict(types.most_common(50)),
            "signals": signals,
            "errors_preview": errors[:3],
            "html": html,
        }

    # -------------------------
    # MAIN PAGE DATA (rendered)
    # -------------------------
    async def fetch_page_data(self, url):
        html = await self._fetch_rendered_html(url)
        text = self._clean_text(html)

        raw_blocks = self._extract_jsonld_blocks(html)
        parsed, errors = self._parse_jsonld_blocks(raw_blocks)

        internal_links = self._extract_internal_links(html, url)
        candidate_service_urls = self._candidate_service_urls(internal_links)

        return text, parsed, errors, candidate_service_urls

    # -------------------------
    # LLM: ANALYZE + FIX/ADVISE
    # -------------------------
    def analyze_and_fix(self, url, text, schema_parse_errors, audit_summary, candidate_service_urls):
        if not text:
            print("⚠️ No text extracted; skipping.")
            return

        chunks = self._simulate_rag_chunking(text)
        sample_chunks = chunks[:3]

        prompt = f"""
You are a "Generative Engine Optimization (GEO) Architect" and a "Technical SEO auditor".

Your job:
(A) Determine page intent and recommend schema improvements.
(B) Diagnose whether the schema is VISIBLE/USABLE to crawlers.
(C) If schema is not visible/usable, recommend TECHNICAL fixes (not just new JSON-LD).
(D) Ensure Service.url points to the correct dedicated service page (not the homepage).
(E) Add the GEO "Entity Bridge" using Wikidata linking via about/sameAs.

--- URL ---
{url}

--- CRAWL VISIBILITY AUDIT (IMPORTANT) ---
- "server" = raw HTML from requests (closer to initial crawler fetch)
- "rendered" = post-JS DOM from headless browser
Audit summary:
{json.dumps(audit_summary, indent=2)}

Interpretation rules:
1) If server.blocks_count == 0 but rendered.blocks_count > 0:
   -> Schema likely injected by JS. Recommend SSR / output JSON-LD in initial HTML.
2) If errors_count > 0 on server or rendered:
   -> JSON-LD invalid JSON. Recommend fixing syntax, one valid JSON per script, no trailing commas.
3) If robots_meta includes "noindex" or canonical points away:
   -> Recommend fixing indexability/canonicalization first.

--- CANDIDATE SERVICE PAGE URLS (INTERNAL LINKS FOUND) ---
{json.dumps(candidate_service_urls[:25], indent=2)}

IMPORTANT RULE (Service URLs):
- When outputting a Service node (e.g., hasOfferCatalog/offers), set Service.url to the MOST RELEVANT dedicated service page.
- Prefer URLs like /services/... that match the service name.
- If the current page IS the service page, set Service.url to this page URL.
- If NO dedicated service page exists, OMIT Service.url (do NOT point everything to the homepage).

--- GEO GOLD STANDARD: ENTITY BRIDGE (WIKIDATA) ---
Hard requirements:
1) Every output JSON-LD MUST include an "about" array that links key concepts to Wikidata via sameAs.
2) Pattern:
   {{ "@type":"Thing", "name":"<Concept>", "sameAs":"https://www.wikidata.org/wiki/<QID>" }}
3) Do NOT output OWL/SKOS/RDF code.
4) Do NOT invent QIDs. If unsure, include the Thing with "name" only and omit sameAs.

Approved Wikidata QIDs you may use:
FinTech (Financial Technology)
	•	Q16319025 – Fintech — financial technology; technologies used in finance and the industry producing these technologies.  ￼

Foreign Exchange / Financial Concepts (suggested concepts)
Wikidata doesn’t have a single specific “foreign exchange trading / FX management” QID that cleanly maps to commercial FX platforms yet, but the following items are closely related areas you can use:
	•	Q1125 – Foreign exchange — the global market for trading national currencies (multiple sources outside this search confirm this is the canonical Wikidata concept for forex). (You can verify on Wikidata itself if needed.)
	•	Q2431195 – Electronic trading platform — generic concept of digital markets/exchange systems. (Likely useful for platforms like an FX aggregator / trading API.)

Financial & Payment Systems
	•	Q20983193 – Financial services — general service category covering services like FX management. (Search on Wikidata for exact QID — this is a typical concept you’ll find.)
	•	Q6310 – Bank — traditional financial institution; useful if your entity relates to banking or embedded banking services.
	•	Q11055 – Broker — intermediary; relevant for FX brokers and liquidity providers.

Embedded Finance & Related
There isn’t a dedicated “embedded finance” QID yet on Wikidata. A suitable approach is to combine broader FinTech + payment + API platform items such as:
	•	Q20983193 – Financial services
	•	Q19835185 – Application programming interface

Where to attach "about":
- Home: Organization/ProfessionalService AND WebPage
- Service page: Service (and optionally WebPage/ServicePage)
- Blog: BlogPosting (and optionally WebPage)

--- SCHEMA PARSE ERRORS (first 3) ---
{json.dumps(schema_parse_errors[:3], indent=2)}

--- PAGE CONTENT (first 3 chunks) ---
{json.dumps(sample_chunks, indent=2)}

TASKS:
1) Page Intent: Home / Service / Blog / Product / About/Contact
2) Visibility Diagnosis: 3-6 bullets
3) Fix Plan: prioritized steps (technical + schema + validation)
4) JSON-LD: output corrected JSON-LD if missing/incomplete; if JS-only, still provide ideal JSON-LD but emphasize technical fix first.

FORMAT:
1) Page Intent:
2) Visibility Diagnosis:
3) Fix Plan:
4) JSON-LD:
"""

        try:
            model = genai.GenerativeModel(self.model_name)
            response = model.generate_content(prompt)
            print("\n" + "=" * 40)
            print(f"🔧 REPORT FOR: {url}")
            print("=" * 40)
            print(response.text)
        except Exception as e:
            print(f"❌ Gemini Error: {e}")

In [ ]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("⚠️ Please set your GEMINI_API_KEY in Colab secrets!")

TARGET_URLS = [
    "https://okoora.com/business/",
    "https://okoora.com/business/who-its-for/",
    "https://okoora.com/business/how-it-works/",
    "https://okoora.com/business/pricing/",
    "https://okoora.com/business-relevant-currencies/"

]

auditor = Universal_GEO_Auditor(GEMINI_API_KEY)
loop = asyncio.get_event_loop()

In [ ]:
for url in TARGET_URLS:
    print(f"\n💠 Processing: {url} ...")

    # server audit (what initial HTML shows)
    server_audit = auditor.schema_audit_server(url)

    # rendered audit (what JS-rendered DOM shows)
    rendered_audit = loop.run_until_complete(auditor.schema_audit_rendered(url))

    # page content + schema parse errors + candidate service URLs
    text, parsed_schema, schema_errors, candidate_service_urls = loop.run_until_complete(
        auditor.fetch_page_data(url)
    )

    # compact audit summary for the LLM
    audit_summary = {
        "server": {
            "blocks_count": server_audit["blocks_count"],
            "errors_count": server_audit["errors_count"],
            "types": server_audit["types"],
            "signals": server_audit["signals"],
            "errors_preview": server_audit["errors_preview"],
        },
        "rendered": {
            "blocks_count": rendered_audit["blocks_count"],
            "errors_count": rendered_audit["errors_count"],
            "types": rendered_audit["types"],
            "signals": rendered_audit["signals"],
            "errors_preview": rendered_audit["errors_preview"],
        },
    }

    auditor.analyze_and_fix(
        url=url,
        text=text,
        schema_parse_errors=schema_errors,
        audit_summary=audit_summary,
        candidate_service_urls=candidate_service_urls
    )

    print("\n" + "-" * 60 + "\n")


💠 Processing: https://okoora.com/business/ ...

🔧 REPORT FOR: https://okoora.com/business/
Here is the GEO and Technical SEO audit for the provided URL.

### 1) Page Intent
**Service Page**

This page (`/business/`) acts as a primary landing page or a top-level service category page for Okoora's business-to-business offerings. Its title, "Business FX Infrastructure," clearly positions it as a hub for understanding the company's core services for corporate clients.

### 2) Visibility Diagnosis
*   **No Schema Detected:** The primary issue is the complete absence of structured data. The audit shows `blocks_count: 0` in both the initial server HTML and the post-JavaScript rendered DOM.
*   **No Crawler Injection:** This is not a client-side rendering issue. The schema is simply missing from the page's source code altogether.
*   **Good Indexability Signals:** The page is indexable. There are no "noindex" directives, and the canonical tag `https://okoora.com/business/` is correctly self-r